# Using InternetDB

## Overview: Gather info on unique IP adresses


---
## Project Workflow:
1. **Step 1: Install and Import Necessary Libraries**
2. **Step 2: Load Data**
3. **Step 3: Analyze Device IPs**



# Step 1: Install and Import Necessary Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import ast
from collections import Counter

AttributeError: module 'collections' has no attribute 'version'

# Step 2: Load Data - Only 2-Model System

In [338]:
model_info = pd.read_csv("data/twoM_devices.csv")

In [339]:
model_info = model_info[model_info['Metric'] == 'Recall']
model_info

,src_mac,Model,Metric,Value
24,AMCREST WiFi Camera,Binary Logistic Regression,Recall,0.971264
25,Amazon Echo Show,Binary Logistic Regression,Recall,0.997498
26,Arlo Q Indoor Camera,Binary Logistic Regression,Recall,1.000000
27,D-Link Mini Camera,Binary Logistic Regression,Recall,1.000000
28,HeimVision Smart WiFi Camera,Binary Logistic Regression,Recall,1.000000
29,Home Eye Camera,Binary Logistic Regression,Recall,1.000000
30,Nest Indoor Camera,Binary Logistic Regression,Recall,0.998818
31,Netatmo Camera,Binary Logistic Regression,Recall,0.999169
32,Rbcior Camera,Binary Logistic Regression,Recall,0.986111
33,TP-Link Tapo Camera,Binary Logistic Regression,Recall,0.998390


In [340]:
model_info = model_info.groupby('src_mac')['Value'].mean().reset_index()

model_info.columns = ['src_mac', 'Recall']
model_info = model_info.rename(columns={'src_mac': 'camera_model'})

model_info

,camera_model,Recall
0,AMCREST WiFi Camera,0.885632
1,Amazon Echo Show,0.984129
2,Arlo Q Indoor Camera,0.995656
3,D-Link Mini Camera,0.911765
4,HeimVision Smart WiFi Camera,0.945130
5,Home Eye Camera,0.926829
6,Nest Indoor Camera,0.971131
7,Netatmo Camera,0.967313
8,Rbcior Camera,0.949451
9,TP-Link Tapo Camera,0.943439


In [341]:
model_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   camera_model  12 non-null     object 
 1   Recall        12 non-null     float64
dtypes: float64(1), object(1)
memory usage: 324.0+ bytes


## load IP Data

In [342]:
IP_data = pd.read_csv("data/IP_data.csv")

In [343]:
IP_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IP            264 non-null    object
 1   open_ports    264 non-null    object
 2   tags          264 non-null    object
 3   vulns         264 non-null    object
 4   camera_model  264 non-null    object
dtypes: object(5)
memory usage: 10.4+ KB


In [ ]:
# fixing the data types
IP_data['tags'] = IP_data['tags'].apply(ast.literal_eval)
IP_data['open_ports'] = IP_data['open_ports'].apply(ast.literal_eval)
IP_data['vulns'] = IP_data['vulns'].apply(ast.literal_eval)

In [345]:
IP_data[IP_data['camera_model'] == 'Rbcior Camera'].sample(5)



,IP,open_ports,tags,vulns,camera_model
27,34.213.103.51,[1883],[cloud],[],Rbcior Camera
8,44.230.212.240,"[80, 443]","[self-signed, cloud]",[],Rbcior Camera
102,52.218.132.217,"[80, 443, 444]",[cloud],[],Rbcior Camera
59,44.231.176.168,"[80, 443]","[self-signed, cloud]",[],Rbcior Camera
123,54.188.99.4,[1883],[cloud],[],Rbcior Camera


# Step 3: Analyze Device IPs

## Merge dfs

In [346]:
IP_data = pd.merge(IP_data, model_info, on='camera_model', how='left')

In [347]:
IP_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   IP            264 non-null    object 
 1   open_ports    264 non-null    object 
 2   tags          264 non-null    object 
 3   vulns         264 non-null    object 
 4   camera_model  264 non-null    object 
 5   Recall        264 non-null    float64
dtypes: float64(1), object(5)
memory usage: 12.5+ KB


In [348]:
IP_data.sample(5)

,IP,open_ports,tags,vulns,camera_model,Recall
170,100.20.197.85,"[80, 443]",[cloud],[],Wyze Camera,0.882441
162,35.163.109.225,[443],[cloud],[],Wyze Camera,0.882441
213,34.242.106.87,"[80, 443]",[cloud],[],Arlo Q Indoor Camera,0.995656
144,52.119.198.2,[443],[cloud],[],Amazon Echo Show,0.984129
8,44.230.212.240,"[80, 443]","[self-signed, cloud]",[],Rbcior Camera,0.949451


## Analysis

### Does number of Open Ports affect recall?

In [349]:
def count_OPs(ops):
    return len(ops)

IP_data['num_OPs'] = IP_data['open_ports'].apply(count_OPs)

IP_data[['camera_model', 'num_OPs']].sample(5)

,camera_model,num_OPs
103,Arlo Q Indoor Camera,2
97,Netatmo Camera,1
249,Amazon Echo Show,1
69,Amazon Echo Show,2
146,Rbcior Camera,3


In [ ]:
correlation, p_value = pearsonr(IP_data['num_OPs'], IP_data['Recall'])

print(f"Pearson coefficient: {correlation}")
print(f"p-value: {p_value}")

Pearson correlation coefficient: -0.10533254125620509
P-value: 0.08762061751214718


!!! no significant negative correlation with number of open ports with recall score

In [ ]:
port_counts = Counter(port for port_list in IP_data['open_ports'] for port in port_list)

print(port_counts)

for port in port_counts.keys():
    IP_data[f'port_{port}'] = IP_data['open_ports'].apply(lambda ports: 1 if port in ports else 0)

for port in port_counts.keys():
    correlation, p_value = pearsonr(IP_data[f'port_{port}'], IP_data['Recall'])
    if p_value < 0.05:
        print(f"Port {port} is significant. p-value: {p_value}, Pearson coefficient: {correlation}")


Counter({443: 235, 80: 183, 1883: 15, 1443: 4, 444: 4, 10001: 3, 53: 3, 8080: 3, 32100: 3, 2121: 1, 5003: 1, 25: 1, 110: 1, 587: 1, 993: 1, 995: 1, 2003: 1, 22: 1, 123: 1, 8006: 1, 9200: 1, 3001: 1, 6000: 1, 8088: 1, 5000: 1, 7415: 1, 26656: 1, 5432: 1, 9109: 1})
Port 10001 has a significant p-value: 0.030497468448190675, Pearson correlation: -0.1331966145043893
Port 8080 has a significant p-value: 0.03957262126773755, Pearson correlation: -0.12676245333964423
Port 32100 has a significant p-value: 0.03957262126773755, Pearson correlation: -0.12676245333964423


#### Results
- Port 10001
    - P2P live streaming on cameras like Wyze
- port 8080
    - port 80 alternative
    - http traffic
    - might suggest web server traffic, proxy server usage, or testing and development activities
- port 32100
    - common for p2p communication for many IoT cameras

- P2P is a way for computers and devices to communicate directly with each other without needing a central server
    - share data directly
    - file sharing


### Does num Vulnurabilities affect recall?

In [352]:
def count_vulns(vulns):
    return len(vulns)

IP_data['num_vulns'] = IP_data['vulns'].apply(count_vulns)

IP_data[['camera_model', 'num_vulns']].sample(5)

,camera_model,num_vulns
90,Amazon Echo Show,0
87,Wyze Camera,0
247,Wyze Camera,0
19,Amazon Echo Show,0
151,Amazon Echo Show,0


In [ ]:
# Perform Pearson correlation test
correlation, p_value = pearsonr(IP_data['num_vulns'], IP_data['Recall'])

# Print the results
print(f"Pearson coefficient: {correlation}")
print(f"p-value: {p_value}")


Pearson correlation coefficient: -0.12440415742583635
P-value: 0.043426063045677435


#### Results
significant negative correlation with number of vulnurabilities with recall score

### Investigate specific vulns on each camera model and what they correspond to



In [ ]:
vulns_count = Counter(vuln for vulns_list in IP_data['vulns'] for vuln in vulns_list)

print(vulns_count)

for vuln in vulns_count.keys():
    IP_data[f'vuln_{vuln}'] = IP_data['vulns'].apply(lambda vulns: 1 if vuln in vulns else 0)

for vuln in vulns_count.keys():
    correlation, p_value = pearsonr(IP_data[f'vuln_{vuln}'], IP_data['Recall'])
    if p_value < 0.05:
        print(f"Vuln {vuln} is significant. p-value: {p_value}")


Counter({'CVE-2011-1176': 5, 'CVE-2012-3526': 5, 'CVE-2013-2765': 5, 'CVE-2007-4723': 5, 'CVE-2012-4360': 5, 'CVE-2013-4365': 5, 'CVE-2013-0942': 5, 'CVE-2009-2299': 5, 'CVE-2009-0796': 5, 'CVE-2011-2688': 5, 'CVE-2012-4001': 5, 'CVE-2013-0941': 5, 'CVE-2023-44487': 5, 'CVE-2021-3618': 5, 'CVE-2021-23017': 5, 'CVE-2019-9516': 4, 'CVE-2019-9511': 4, 'CVE-2019-9513': 4, 'CVE-2019-20372': 4, 'CVE-2018-1302': 4, 'CVE-2018-1303': 4, 'CVE-2022-22720': 4, 'CVE-2021-39275': 4, 'CVE-2022-28614': 4, 'CVE-2021-40438': 4, 'CVE-2021-32786': 4, 'CVE-2022-29404': 4, 'CVE-2021-32792': 4, 'CVE-2022-28615': 4, 'CVE-2016-8612': 4, 'CVE-2021-32785': 4, 'CVE-2021-44790': 4, 'CVE-2022-37436': 4, 'CVE-2023-31122': 4, 'CVE-2018-1301': 4, 'CVE-2006-20001': 4, 'CVE-2021-34798': 4, 'CVE-2017-9798': 4, 'CVE-2021-32791': 4, 'CVE-2022-31813': 4, 'CVE-2022-28330': 4, 'CVE-2024-40898': 4, 'CVE-2022-22721': 4, 'CVE-2022-30556': 4, 'CVE-2022-22719': 4, 'CVE-2019-11358': 3, 'CVE-2020-11023': 3, 'CVE-2020-11022': 3, 'CVE

#### Results

| Year | Significant Vulnerabilities |
|------|----------------|
| 2016 | 8743, 4975, 4979, 5387, 1546, 8740 |
| 2017 | 9788, 3169, 7679, 15710, 3167, 15715 |
| 2018 | 17199, 1312, 1333, 1283, 17189, 11763 |
| 2019 | 0196, 0211, 10082, 0217, 10098, 10092, 0220, 17567 |
| 2020 | 35452, 1927, 1934, 13938, 11985 |
| 2021 | 26690, 33193, 44224, 26691 |
| 2022 | 36760, 23943, 26377 |
| 2023 | 45802, 25690 |
| 2024 | 38474, 38476, 27316, 38477 |

### Are any tags associated with lower recalls?

In [354]:
tag_counts = Counter(tag for tags_list in IP_data['tags'] for tag in tags_list)

print(tag_counts)

Counter({'cloud': 254, 'cdn': 28, 'self-signed': 8, 'eol-product': 8, 'starttls': 1, 'database': 1})


In [355]:
for tag in tag_counts.keys():
    IP_data[f'tag_{tag}'] = IP_data['tags'].apply(lambda tags: 1 if tag in tags else 0)


IP_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 43 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   IP               264 non-null    object 
 1   open_ports       264 non-null    object 
 2   tags             264 non-null    object 
 3   vulns            264 non-null    object 
 4   camera_model     264 non-null    object 
 5   Recall           264 non-null    float64
 6   num_OPs          264 non-null    int64  
 7   port_80          264 non-null    int64  
 8   port_443         264 non-null    int64  
 9   port_1443        264 non-null    int64  
 10  port_1883        264 non-null    int64  
 11  port_10001       264 non-null    int64  
 12  port_2121        264 non-null    int64  
 13  port_5003        264 non-null    int64  
 14  port_53          264 non-null    int64  
 15  port_25          264 non-null    int64  
 16  port_110         264 non-null    int64  
 17  port_587        

In [ ]:
# Ai ----------------------------------------
print("Cloud")
correlation, p_value = pearsonr(IP_data['tag_cloud'], IP_data['Recall'])

print(f"Pearson correlation coefficient: {correlation}")
print(f"P-value: {p_value}")

print("self_signed")
correlation, p_value = pearsonr(IP_data['tag_self-signed'], IP_data['Recall'])

print(f"Pearson correlation coefficient: {correlation}")
print(f"P-value: {p_value}")

print("cdn")
correlation, p_value = pearsonr(IP_data['tag_cdn'], IP_data['Recall'])

print(f"Pearson correlation coefficient: {correlation}")
print(f"P-value: {p_value}")

print("eol_product")
correlation, p_value = pearsonr(IP_data['tag_eol-product'], IP_data['Recall'])

print(f"Pearson correlation coefficient: {correlation}")
print(f"P-value: {p_value}")

print("starttls")
correlation, p_value = pearsonr(IP_data['tag_starttls'], IP_data['Recall'])

print(f"Pearson correlation coefficient: {correlation}")
print(f"P-value: {p_value}")


print("database")
correlation, p_value = pearsonr(IP_data['tag_database'], IP_data['Recall'])

print(f"Pearson correlation coefficient: {correlation}")
print(f"P-value: {p_value}")


Cloud
Pearson correlation coefficient: 0.08071101076494513
P-value: 0.19110757461343303
self_signed
Pearson correlation coefficient: -0.026894017290070415
P-value: 0.6635760627028047
cdn
Pearson correlation coefficient: 0.32157187939883064
P-value: 9.156699138662372e-08
eol_product
Pearson correlation coefficient: -0.11800765722561866
P-value: 0.0554930876588917
starttls
Pearson correlation coefficient: -0.07290753045365297
P-value: 0.23777847169932306
database
Pearson correlation coefficient: 0.05756747425513874
P-value: 0.3514940810155973


#### Results
- eol product is close to 0.05 alpha but not less, none are significantly correlated at 0.05 level
- eol product is end of life product
    - no longer recieve updates from vendor
        - unpatched vulns